In [1]:
"""
Created on Fri Nov 19 14:16:09 2021

@author: sjohnstone
"""

#Import libraries for requesting downloads from websites, and for navigating operating system file structure
import urllib
import os
from glob import glob

import numpy as np

import pointCloudCreation as pcc #Some wrappers for PDAL pipeline creation and execution

#Specify the path to where we want to save things
savePath = 'D:\\ResearchProjects\\Blanca_lidar\\Blanca_LAS_test\\tiny_request' #Where do we want to store all the files we download?

#Make the directory if it doesnt exist
if not(os.path.isdir(savePath)):
    os.mkdir(savePath)



In [2]:
#When gridding up data we can specify extents and projections if we want to transform and crop the data

#We could also use this information to obtain the list of files using the national maps APIs
# https://apps.nationalmap.gov/help/documents/TNMAccessAPIDocumentation/TNMAccessAPIDocumentation.pdf 

#Lets grid this up
extent = ([-105.578, -105.544],[37.630, 37.604]) #([MinX,MaxX],[MinY,MaxY])z
extent_epsg = 4326 #SRS for extent
out_epsg = 32613 #SRS for output - this is WGS84 UTM Zone 13 N
out_extent = pcc.reproject_extent(extent,extent_epsg,out_epsg)

**Batch downloading from the national map**

Here we will look at two examples of batch downloading from The National Map; one with lidar data, a second with gridded data.

**Downloading LAS data with a Special condition**

This particular region of interest has data sourced from three collections; one in 2011 and two in 2020. The 2011 data is quite out of date, and superceeded by the 2020 data (change detection anyone?). For just making a DEM, we are going to ignore the 2011 data (so don't want to download its pointcloud).  We are going to insert a special condition to *not* download files that are associated with this collection, by checking each download path in the file we got from TNM. 

In [3]:
#BEWARE! This can download a huge amount of data!
pathToTNMList = 'batch_download_tiny_example.txt'

#Open the path file for reading
with open(pathToTNMList,'r') as f:
    #For each line (a path to a file in the TNM download list)
    for line in f:
        #Strip off any whitespace
        line = line.strip()
        #Get the 'name' of this file as the end of the filepath, we'll use this to save the file
        name = line.split('/')[-1]
        
        #Here we will introduce our special condition, making sure we don't have anything that contains the 
        #name of the old collection. If you didn't want this you'd have to remove this conditional statement.
        if not ('CO_San-Luis-Valley_2011' in line):
            #Keep track of progress
            print('Working on: {}'.format(name))
            urllib.request.urlretrieve(line,os.path.join(savePath,name))

Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_4962.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_4963.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_4964.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_5062.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_5063.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_5064.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_5162.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_5163.laz
Working on: USGS_LPC_CO_SanLuisJuanMiguel_2020_D20_13S_DB_5164.laz


**Using PDAL to merge / grid the data**

We can use PDAL to merge and grid the data. However, PDAL uses a moving circular window to aggregate the point cloud onto a DEM. In this example (where there is actually a fairly low density of ground returns in places) this does not seem to produce as clean of results as you mike get from creating a TIN using something like lastools.


In [5]:
######## Lets define some things for creating a DEM from the point cloud #########
in_epsg = 1 #For now this doesn't matter - it just needs to be different than out_epsg to reproject the point cloud
grid_name = 'small_test_rad1p4_tileTest' #What will we save the DEM as?
las_directory = savePath #Where are all the '.la (s/z) files we want to work with?
cell_size = 1.0 #What pixel size do we want
radius = cell_size*np.sqrt(2) #The radius of the window used by PDALs gridding to aggregate the point cloud onto a grid
gridding_method = 'idw' #'idw', 'mean', 'min', 'max', 'count', 'std' see: https://pdal.io/stages/writers.gdal.html
doReclassify = False #Do we want to reclassify the point cloud?

#Build the 'pipeline' for PDAL - this is the series of processing steps
pipeline = pcc.gather_las_files_build_dem_creation_pipeline(in_epsg,out_epsg,grid_name,las_directory,cell_size = cell_size,
                           radius = radius,nodatavalue = -9999, doReclassify = doReclassify,doSavePointCloud = False,
                                        pointResolution = None,gridding_method = gridding_method)

pcc.run_pipeline(pipeline)

True

**PDAL also allows you to tile the datasets you are creating**

This can be helpful if you want to limit the size of the DEMs you are creating, however the way I have things implemented herepdal will still load all the point cloud data files in the specified path into memory and uncompress them (so they will take up a lot more ram then disk space)

In [6]:
#Tiling test
grid_name = 'tile_test'
tile_length = 2000.0 #Dimension of each square tile
buffer = 10 # Amount of overlap between each tile
cell_size = 2.0 #What do we want the pixel size to be
radius = cell_size*np.sqrt(2) #What area to we want to look in when gridding?
gridding_method = 'idw' #'idw', 'mean', 'min', 'max', 'count', 'std' see: https://pdal.io/stages/writers.gdal.html

doReclassify = False
doSavePointCloud = False

#Build the 'pipeline' for PDAL - this is the series of processing steps
pipeline = pcc.gather_las_files_build_dem_tile_creation_pipeline(in_epsg,out_epsg, grid_name, las_directory, 
                                                                 tile_length = tile_length, buffer = buffer,
                                                                 cell_size = cell_size, radius = radius,
                                                                 nodatavalue = -9999,
                                                                doReclassify = doReclassify,
                                                                 doSavePointCloud = doSavePointCloud)

#Run the pipeline
pcc.run_pipeline(pipeline)

True

**We can download DEM data in the same way we did above**


In [7]:
#BEWARE! This can download a huge amount of data!
pathToTNMList = 'batch_download_tiny_example_dem.txt'

#Open the path file for reading
with open(pathToTNMList,'r') as f:
    #For each line (a path to a file in the TNM download list)
    for line in f:
        #Strip off any whitespace
        line = line.strip()
        #Get the 'name' of this file as the end of the filepath, we'll use this to save the file
        name = line.split('/')[-1]
        
        #Keep track of progress
        print('Working on: {}'.format(name))
        urllib.request.urlretrieve(line,os.path.join(savePath,name))

Working on: USGS_1M_13_x44y417_CO_SanLuisJuanMiguel_2020_D20.tif
Working on: USGS_1M_13_x45y417_CO_SanLuisJuanMiguel_2020_D20.tif


**We can use GDAL to merge, crop, and reproject these DEMS**

In [8]:
#if you want to search a directory other than the one this notebook is in, prepend a path (e.g., os.path.join(path//to//folder,'*.tif'))
pathToTifs = os.path.join(savePath,'*.tif') #Get the names of the files of interest (could alternatively save this above)
outputName = 'mergedDEMs.tif' #How do we want to name the mer
pixel_size = 1.0
files = glob(pathToTifs)

#This is a wrapper on some GDAL functions
pcc.merge_warp_dems(files,outputName,out_extent,out_epsg,pixel_size)